In [1]:
import cv2
import numpy as np
import os
import  matplotlib.pyplot as plt

from config import Config
from test_csp import test, test_all_datasets
from deepsort import DeepsortTracker
from util import draw_bboxes

from tasks.task2 import tracking_box
from tasks.task3 import hc, get_group_boxes

In [2]:
all_dataset = False
config = Config()
checkpoints = ['100.pth', '200.pth', '300.pth', '400.pth', '500.pth', '600.pth', '700.pth', '794.pth']
output_dir = './testset_output'


if all_dataset:
    img_seq, boxes = test_all_datasets(checkpoints)
    allseqresult_file = './detection_result'
    try:
        with np.load(allseqresult_file, allow_pickle=True) as dataset:
            img_seq = dataset['img_seq']
            boxes = dataset['boxes']
    except:
        # load from dir
        img_seq, boxes = test_all_datasets(checkpoints)
        np.savez(allseqresult_file, img_seq=img_seq, boxes=boxes)
else: 
    img_seq, boxes = test()

Use cuda: True
Loading pretrained model from ./checkpoints/CSP_epoch_10.pth


In [3]:
task1_save_to_file = True
task2_save_to_file = True
task3_save_to_file = True

In [4]:
# Task 1

tracker = DeepsortTracker(config)
# tracking
seq_boxes = []
seq_instances = []
seq_states = []

track_seq = {}
for i in range(len(img_seq)):
    
    img = img_seq[i].copy() 
    # The boxes have its confidence
    curr_boxes = boxes[i]
    curr_boxes_x1y1x2y2, curr_instances, curr_boxes_state = tracker.detect(img, curr_boxes)
    if task1_save_to_file and curr_boxes_x1y1x2y2 != []:
        img = draw_bboxes(img, curr_boxes_x1y1x2y2, curr_instances)
    for j in range(len(curr_boxes)):
        box = curr_boxes_x1y1x2y2[j]
        instance = curr_instances[j]
        if instance not in track_seq:
            track_seq[instance] = []
        p = ((box[0] + box[2]) // 2, box[3])
        track_seq[instance].append(p)
        k = 1
        track_frames = track_seq[instance][max(0, len(track_seq[instance]) - 20):]
        p_beg = track_frames[0]
        while k < len(track_frames):
            p_end = track_frames[k]
            cv2.line(img, p_beg, p_end, (0, 0, 0))
            p_beg  = p_end
            k = k + 1
    cv2.imwrite(os.path.join(output_dir, 'task1', str(i).zfill(5) + '.jpg'), img)
    seq_boxes.append(curr_boxes_x1y1x2y2)
    seq_instances.append(curr_instances)
    seq_states.append(curr_boxes_state)

Loading weights from ./checkpoints/Deepsort_ckpt.t7... Done!


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


In [ ]:
# Task 2
def task2(count_box):
    task2_output_seq = tracking_box(img_seq, seq_boxes, seq_instances, seq_states, count_box)
    if task2_save_to_file:
        for i in range(len(task2_output_seq)):
            img = task2_output_seq[i]
            cv2.imwrite(os.path.join(output_dir, 'task2', str(i).zfill(5) + '.jpg'), img)
        

In [ ]:
x1 = 500
y1 = 200
x2 = 700
y2 = 400
count_box = [x1, y1, x2, y2]

task2(count_box)

In [ ]:
# Task3 Group detection

def task3():
    for i in range(len(seq_boxes)):
        img = img_seq[i].copy()
        boxes = seq_boxes[i]
        states = seq_states[i]
        instances = seq_instances[i]
        img = draw_bboxes(img, boxes, instances)
        clusters = hc(boxes, states, instances, 10.0, 1)
        clusters_boxes = [[] for i in range(len(boxes))]
        for j in range(len(boxes)):
            clusters_boxes[clusters[j]].append(boxes[j])
        group_boxes = []
        for j in range(len(boxes)):
            if len(clusters_boxes[j]) > 1:
                group_box = get_group_boxes(clusters_boxes[j])
                group_boxes.append(group_box)
        img = draw_bboxes(img, group_boxes)
        # draw group boxes 
        
        
        if task3_save_to_file:
            cv2.imwrite(os.path.join(output_dir, 'task3', str(i).zfill(5) + '.jpg'), img)

In [ ]:
task3()